In [ ]:
from collections import defaultdict
import os
import cv2
import numpy as np
import random

In [ ]:
classes = ['A', 'B1', 'B2', 'B4', 'B5', 'B6', 'G']
target_count_per_class = 75

path = r"C:\Users\Olenka\PycharmProjects\violence_detection\train"
class_counts = defaultdict(int)
video_paths_by_class = defaultdict(list)

In [ ]:
def extract_labels(file):
    parts = file[:-4].split("label_")
    if len(parts) < 2:
        return [0] * len(classes)
    return [1 if cls in parts[1].split('-') else 0 for cls in classes]


In [ ]:
for file in os.listdir(path):
    if file.endswith('.mp4'):
        labels = extract_labels(file) 
        if sum(labels) == 1:
            label_index = labels.index(1)
            cls = classes[label_index]
            class_counts[cls] += 1
            video_paths_by_class[cls].append(os.path.join(path, file))

In [ ]:
class_counts

In [ ]:
def add_noise(frame, level=5):
    noise = np.random.normal(0, level, frame.shape).astype(np.int16)
    frame_int = frame.astype(np.int16)
    noisy = frame_int + noise
    noisy = np.clip(noisy, 0, 255).astype(np.uint8)
    return noisy

In [ ]:
def adjust_brightness(frame, level=0.2):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:, :, 2] = hsv[:, :, 2] * (1 + level)
    hsv[:, :, 2][hsv[:, :, 2] > 255] = 255
    hsv = np.array(hsv, dtype=np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

In [ ]:
def apply_blur(frame, sigma=0.5):
    ksize = int(2 * round(3 * sigma) + 1)
    return cv2.GaussianBlur(frame, (ksize, ksize), sigma)

In [ ]:
def adjust_contrast(frame, level=30):
    alpha = 1.0 + (level / 100.0) 
    beta = 0
    try:
        return cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)
    except Exception as e:
        print(f"Error adjusting contrast: {e}")
        return frame

In [ ]:
def horizontal_flip(frame):
    return cv2.flip(frame, 1)

In [ ]:
augmentations = [
    lambda frame: add_noise(frame, level=random.choice([15, 25, 35])),
    lambda frame: adjust_brightness(frame, level=random.choice([-0.4, -0.2, 0.2, 0.4])),
    lambda frame: apply_blur(frame, sigma=random.choice([0.1, 0.3, 0.5])),
    lambda frame: adjust_contrast(frame, level=random.randint(-50, 50)),
    horizontal_flip
]

def get_augmentation_techniques():
  number = random.randint(2, 4)
  number_of_techniques = random.sample([i for i in range(5)], k=number)
  return number_of_techniques

In [ ]:
def augment(target_class):
    output_dir = path

    augmented_videos = defaultdict(list)
    
    for cls in classes:
        if cls != target_class:
            continue
    
        current_count = class_counts[cls]
        if current_count >= target_count_per_class:
            continue
    
        videos = video_paths_by_class[cls]
        i = 0
        while current_count + len(augmented_videos[cls]) < target_count_per_class:
            original_path = videos[i % len(videos)]
    
            cap = cv2.VideoCapture(original_path)
            fps = cap.get(cv2.CAP_PROP_FPS)
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
            frames = []
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame)
            cap.release()
    
            aug_indices = get_augmentation_techniques()
            selected_augs = [augmentations[idx] for idx in aug_indices]
    
            aug_frames = []
            for frame in frames:
                for aug in selected_augs:
                    frame = aug(frame)
                aug_frames.append(frame)
    
            filename = f"{i}-aug_label_{cls}-0-0.mp4"
            output_path = os.path.join(output_dir, filename)
    
            out = cv2.VideoWriter(
                output_path,
                cv2.VideoWriter_fourcc(*'mp4v'),
                fps,
                (width, height)
            )
    
            for frame in aug_frames:
                frame_bgr = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)
                out.write(frame_bgr)
            out.release()
    
            augmented_videos[cls].append(output_path)
            i += 1
            print(f"Saved: {filename}")


In [ ]:
augment("B4")
augment("B5")